# 7.2. Случайный лес

Ваша задача - написать класс `random_forest` для решения задачи классификации на основе датасета Ирисов Фишера (`sklearn.datasets.load_iris`), принимающий на вход конструктора аргументы `n_estimators`, `max_depth`, `subspaces_dim` и `random_state`. описание этих аргументов приведено ниже. У этого класса должны быть определены методы `.fit()` и `.predict()`, а также поле `._estimators`, в котором должен храниться список алгоритмов, используемых в ансамбле.

- n_estimators - число деревьев в ансамбле
- max_depth - максимальная глубина каждого дерева в ансамбле
- subspaces_dim - размерность случайного подпространства для каждого дерева

Для создания обучающей подвыборки для каждого из базовых классификаторов, Вы можете воспользоваться классом `sample`, который Вы реализовали в прошлом задании. В случае его использования, не забудьте включить его описание в файл с Вашим решением текущего задания. Мы также предлагаем вам организовать выбор подпространств для каждого дерева посредством заполнения списка `subspace_idx`, в котором будут логироваться выбранные для каждого базового классификатора подпространства.

Замечание: в рамках выполнения данного задания запрещено использовать класс `sklearn.ensemble.RandomForestClassifier`. Такой код не пройдёт проверку.

Подберите также гиперпараметры, на которых ваш алгоритм получает наилучшее качество (с точки зрения метрики accuracy, доли правильных ответов) на тестовой выборке с параметром `test_size`=0.3, задайте их в виде глобальных переменных N_ESTIMATORS, MAX_DEPTH, SUBSPACE_DIM.

Шаблон класса:

## Примечания

1. В данной задаче запрещено использовать библиотеку pandas.

2. В реализуемых методах запрещается использовать вывод любой информации на экран (в частности, недопустимо использование print()).

# Пример входных и выходных данных

In [19]:
import numpy as np

a = np.array([[1,2,3,1,2,2],[1,2,3,4,2,2]])
np.argmax(np.bincount(a[:,0]))

1

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.tree import DecisionTreeClassifier

np.random.seed(42)

# Загрузка датасета Iris
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.3)

import numpy as np
from sklearn.tree import DecisionTreeClassifier

np.random.seed(42)

class sample:
    def __init__(self, X, n_subspace):
        self.n_subspace = n_subspace
        self.idx_subspace = self.random_subspace(X, n_subspace)
    
    def __call__(self, X, y):
        idx_obj = self.bootstrap_sample(X)
        X_sampled, y_sampled = self.get_subsample(X, y, self.idx_subspace, idx_obj)
        return X_sampled, y_sampled

    @staticmethod
    def bootstrap_sample(X):
        n_samples = X.shape[0]
        indices = np.random.choice(range(n_samples), size=n_samples, replace=True)
        return indices

    @staticmethod
    def random_subspace(X, n_subspace):
        n_features = X.shape[1]
        return np.random.choice(range(n_features), size=n_subspace, replace=False)

    @staticmethod
    def get_subsample(X, y, idx_subspace, idx_obj):
        X_sampled = X[idx_obj][:, idx_subspace]
        y_sampled = y[idx_obj]
        return X_sampled, y_sampled

N_ESTIMATORS = 40
MAX_DEPTH = 7
SUBSPACE_DIM = 2 

class random_forest:
    def __init__(self, n_estimators, subspaces_dim, max_depth, random_state=42):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.subspaces_dim = subspaces_dim
        self.random_state = random_state
        np.random.seed(random_state)
        self._estimators = [
            DecisionTreeClassifier(max_depth=max_depth, random_state=random_state + i)
            for i in range(self.n_estimators)
        ]
        self.samples = []
    
	def counts(preds: np.array) -> list:
        pred = []
        
		for 

        return pred

    def fit(self, X, y):
        for i in range(self.n_estimators):
            samp = sample(X, self.subspaces_dim)
            samp_X, samp_y = samp(X, y)
            self._estimators[i].fit(samp_X, samp_y)
            self.samples.append(samp)

    def predict(self, X):
        preds = []
        for i, estimator in enumerate(self._estimators):
            subspace_idx = self.samples[i].idx_subspace
            X_transformed = X[:, subspace_idx]
            preds.append(estimator.predict(X_transformed))

		pred =     
		
        return 


forest = random_forest(n_estimators=N_ESTIMATORS, subspaces_dim=SUBSPACE_DIM, max_depth=MAX_DEPTH)
forest.fit(X_train, y_train)
y_pred = forest.predict(X_test)
y_pred

[array([1, 0, 2, 1, 1, 0, 1, 2, 2, 1, 2, 0, 0, 0, 0, 2, 2, 1, 1, 2, 0, 2,
        0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 2, 2, 1, 0,
        0]),
 array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
        0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 2, 1, 1, 0,
        0]),
 array([1, 0, 1, 1, 2, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 1,
        0, 2, 2, 2, 1, 2, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0,
        0]),
 array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
        0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 2, 1, 1, 0,
        0]),
 array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
        0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 2, 1, 1, 0,
        0]),
 array([1, 0, 2, 1, 2, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 1,
        0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 2, 1, 1, 0,
        0]),
 array([1, 0, 2, 1, 2, 0, 1, 2, 2,

In [2]:
from sklearn.ensemble import RandomForestClassifier

trees = RandomForestClassifier(n_estimators= 25, max_depth= 15, random_state= 42 )

trees.fit(X_train, y_train)
trees.predict(X_test)

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 2, 1, 1, 0,
       0])